In [3]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

In [5]:
import os
from langchain_groq import ChatGroq

TEMPERATURE=0.5
LLM_MODEL_NAME="mixtral-8x7b-32768"
EMBED_MODEL_NAME="sentence-transformers/all-MiniLM-L6-v2"
MILVUS_URI="./milvus_example.db"

In [6]:
llm = ChatGroq(temperature=TEMPERATURE, model_name=LLM_MODEL_NAME, api_key=os.getenv("GROQ_API_KEY"))
llm.invoke("hi")

AIMessage(content="Hello! How can I help you today? If you have any questions or need assistance with package tracking, shipping rates, or anything else related to USPS, please let me know. I'm here to help!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_time': 0.067878101, 'prompt_time': 0.001768355, 'queue_time': 0.020904363000000002, 'total_time': 0.069646456}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc831ccd-0352-41f2-bb07-8e58e9bd900d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 45, 'total_tokens': 53})

In [ ]:
llm.invoke("hi")

In [15]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [20]:
res = model.invoke("Hello, world!")

In [24]:
d = res.response_metadata["token_usage"]

In [25]:
from logger import graph_logger as logger

logger.debug('Token usage: %s', d)

2025-02-26 09:18:43,953 | DEBUG | graph | Token usage: {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_time': 0.008333333, 'prompt_time': 0.002980471, 'queue_time': 0.015748617, 'total_time': 0.011313804}


In [26]:
logger.debug('{0}'.format(d))

2025-02-26 09:19:34,022 | DEBUG | graph | {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_time': 0.008333333, 'prompt_time': 0.002980471, 'queue_time': 0.015748617, 'total_time': 0.011313804}


In [ ]:
from langchain.schema import HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
AIMessage(content="Hi i am an AI.").content

In [7]:

import os
import sqlite3
import configparser

from langchain_core.prompts import PromptTemplate
from typing import Any, Literal
from models import ChatState, Grade
from abc import ABC, abstractmethod
from langchain_groq import ChatGroq
from vector_store import vector_store
from langchain.schema import SystemMessage
from logger import graph_logger as logger
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.tools import TavilySearchResults

In [19]:

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")
structured_llm = llm.with_structured_output(Grade)
prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user query. \n 
    Here is the retrieved document: \n\n {context} \n\n
    Here is the user query: {query} \n
    If the document contains keyword(s) or semantic meaning related to the user query, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the query.""",
    input_variables=["context", "query"],
)
chain = prompt | structured_llm

scored_result = chain.invoke({"query": "slowness", "context": "slow as a turtle"})
#score = scored_result.binary_score
scored_result

Grade(binary_score='yes')

In [18]:
scored_result.binary_score

'yes'